In [1]:
import numpy as np
import string
from pyvi import ViTokenizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from scipy.spatial import distance

In [2]:
# Pre-processing

In [5]:
import os
dir = 'data/chatbot'
filenames = os.listdir(dir)
print(filenames)

['bạn bè.txt', 'các câu hỏi phức tạp.txt', 'du lịch.txt', 'gia đình.txt', 'giải trí.txt', 'học tập.txt', 'nghề nghiệp.txt', 'nghỉ lễ.txt', 'người yêu.txt', 'robot.txt', 'shoping.txt', 'sở thích.txt', 'tdtu.txt', 'thông tin cá nhân.txt', 'trò chuyện về đi ăn.txt', 'tán gẫu.txt', 'đất nước.txt', 'địa chỉ.txt']


In [9]:
Q = []
A = []
for i in filenames:
    with open(dir+'/'+i,encoding="utf8") as f:
        for s in f:
            a = s.split('__eou__')
            Q.append(a[0])
            A.append(a[1])
print('Number of QA pairs:', len(Q))

Number of QA pairs: 5855


In [15]:
# Use Word2vec

In [16]:
sents = []
for i in range(len(Q)):
    tokens = ViTokenizer.tokenize(Q[i])
    sents.append(tokens)
    
modelw2v = Word2Vec(sentences=sents, size=100, window=5, min_count=2, sg=1)

In [17]:
# Calculate the vector average of each question
def avg_sentence_vector(words, model, num_features, index2word_set):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0
    
    tokens = ViTokenizer.tokenize(words)
    for word in words:
        if word in index2word_set:
            nwords = nwords+1
            featureVec = np.add(featureVec, model.wv[word])

    if nwords > 0:
        featureVec = np.divide(featureVec, nwords)
    return featureVec

In [18]:
def QuestionAnswer(s):
    sentence_1_avg_vector = avg_sentence_vector(s, model=modelw2v, num_features=100, index2word_set= modelw2v.wv.index2word)

    max_ans = 1
    for sentence_2 in Q:
        sentence_2_avg_vector = avg_sentence_vector(sentence_2, model=modelw2v, num_features=100, index2word_set= modelw2v.wv.index2word)

        values =  distance.cosine(sentence_1_avg_vector,sentence_2_avg_vector)
        if(values < max_ans):
            max_ans = values
            best_ans = A[Q.index(sentence_2)]
            best_que = sentence_2
            print(best_ans)
    return best_ans

In [19]:
s= "Có môn thể thao nào bạn thích chơi không?"
print(QuestionAnswer(s))

 Ngon nhào vô 
 Mình không thích hút thuốc, vì nó làm hại phổi và răng 
 Thật ra học bài đối với mình là chuyện thường xuyên, có những lúc mình thích, có những lúc mình không thích lắm 
 mình cũng thường hay đi chơi thể thao 
 mình và thể thao là hai đường thẳng song song bạn nha 
 tôi thường chạy bộ vào những lúc rảnh rỗi 
 mình thích chơi đá bóng và billard 
 mình thích chơi đá bóng và billard 


In [10]:
# Not using word2vec version

In [11]:
def cleaner(x):
    return [a for a in (''.join([a for a in x if a not in string.punctuation])).lower().split()]

In [12]:
Pipe = Pipeline([
    ('bow', CountVectorizer(analyzer=cleaner)),
    ('tfidf', TfidfTransformer()),
    ('classifier', DecisionTreeClassifier())
])

In [13]:
Pipe.fit(Q, A)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function cleaner at 0x0000023ADEC66700>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', DecisionTreeClassifier())])

In [14]:
Pipe.predict(['Có môn thể thao nào bạn thích chơi không?'])[0]

' tôi thường chạy bộ vào những lúc rảnh rỗi '